In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from loguru import logger
from gwpy import signal, timeseries


from pygwb.notch import StochNotch, StochNotchList
from pygwb.util import calc_bias
from pygwb import baseline, parameters
from pygwb.delta_sigma_cut import run_dsc

/opt/hostedtoolcache/Python/3.8.18/x64/lib/python3.8/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


# Produce a $\Delta\sigma$ cut

In [2]:
def dsc_cut(
    naive_sigma: np.ndarray,
    slide_sigma: np.ndarray,
    dsc: float = 0.2,
    bf_ss: float = 1,
    bf_ns: float = 1,
):
    dsigma = np.abs(slide_sigma * bf_ss - naive_sigma * bf_ns) / slide_sigma * bf_ss

    return dsigma >= dsc, dsigma

In [3]:
def calc_Hf(freqs: np.ndarray, alpha: float = 0, fref: int = 20):
    Hf = (freqs / fref) ** alpha
    return Hf  # do for different power laws , take all badgps times from all alphas, multiple calls in main func

In [4]:
def calc_sigma_alpha(sensitivity_integrand_with_Hf: np.ndarray):
    sigma_alpha = np.sqrt(1 / np.sum(sensitivity_integrand_with_Hf))
    return sigma_alpha

In [5]:
def WindowFactors(window1: np.ndarray, window2: np.ndarray):
    N1 = len(window1)
    N2 = len(window2)
    Nred = np.gcd(N1, N2).astype(int)
    indices1 = (np.array(range(0, Nred, 1)) * N1 / Nred).astype(int)
    indices2 = (np.array(range(0, Nred, 1)) * N2 / Nred).astype(int)
    window1red = window1[indices1]
    window2red = window2[indices2]

    # extract 1st and 2nd half of windows

    cut = int(np.floor(Nred / 2))

    firsthalf1 = window1red[0:cut]
    secondhalf1 = window1red[cut:Nred]

    firsthalf2 = window2red[0:cut]
    secondhalf2 = window2red[cut:Nred]

    # calculate window factors
    w1w2bar = np.mean(window1red * window2red)
    w1w2squaredbar = np.mean((window1red**2) * (window2red**2))
    w1w2ovlsquaredbar = np.mean((firsthalf1 * secondhalf1) * (firsthalf2 * secondhalf2))

    return w1w2bar, w1w2squaredbar, w1w2ovlsquaredbar

In [6]:
def calc_sens_integrand(
    freq: np.ndarray,
    P1: np.ndarray,
    P2: np.ndarray,
    window1: np.ndarray,
    window2: np.ndarray,
    delta_f: float,
    orf: np.array,
    T: int = 32,
    H0: float = 67.9e3 / 3.086e22,
):
    w1w2bar, w1w2squaredbar, oo = WindowFactors(window1 = window1, window2 = window2)
    S_alpha = 3 * H0**2 / (10 * np.pi**2) * 1.0 / freq**3
    sigma_square_avg = (
        (w1w2squaredbar / w1w2bar**2)
        * 1
        / (2 * T * delta_f)
        * P1
        * P2
        / (orf**2.0 * S_alpha**2)
    )

    return sigma_square_avg

In [7]:
def veto_lines(freqs: np.ndarray, lines: np.ndarray, df: float = 0):
    nbins = len(freqs)
    veto = np.zeros((nbins, 1), dtype="bool")

    if not len(lines):
        return veto

    fmins = lines[:, 0]
    fmaxs = lines[:, 1]
    for fbin in range(len(freqs)):
        freq = freqs[fbin]
        index = np.argwhere((freq >= (fmins - df)) & (freq <= fmaxs + df))
        if index.size != 0:
            veto[fbin] = True
    return veto
    

In [8]:
def run_dsc(
    dsc: float,
    segment_duration: int,
    sampling_frequency: int,
    psd1_naive: np.ndarray,
    psd2_naive: np.ndarray,
    psd1_slide: np.ndarray,
    psd2_slide: np.ndarray,
    alphas: np.ndarray,
    orf: np.array,
    notch_list_path: str = "",
):
    if notch_list_path:
        lines_stochnotch = StochNotchList.load_from_file(f"{notch_list_path}")
        lines = np.zeros((len(lines_stochnotch), 2))

        for index, notch in enumerate(lines_stochnotch):
            lines[index, 0] = lines_stochnotch[index].minimum_frequency
            lines[index, 1] = lines_stochnotch[index].maximum_frequency
    else:
        lines = np.zeros((0, 2))

    logger.info("Running delta sigma cut")
    nalphas = len(alphas)
    times = np.array(psd1_naive.times)
    ntimes = len(times)
    df = psd1_naive.df.value
    dt = psd1_naive.df.value ** (-1)
    bf_ns = calc_bias(segmentDuration = segment_duration, deltaF = df, deltaT = dt, N_avg_segs=1)  # Naive estimate
    bf_ss = calc_bias(segmentDuration = segment_duration, deltaF = df, deltaT = dt, N_avg_segs=2)  # Sliding estimate
    freqs = np.array(psd1_naive.frequencies)
    overall_cut = np.zeros((ntimes, 1), dtype="bool")
    cuts = np.zeros((nalphas, ntimes), dtype="bool")
    dsigmas = np.zeros((nalphas, ntimes), dtype="bool")
    veto = veto_lines(freqs = freqs, lines = lines)
    keep = np.squeeze(~veto)

    window1 = np.hanning(segment_duration * sampling_frequency)
    window2 = window1
    for alpha in range(nalphas):
        Hf = calc_Hf(freqs = freqs, alpha = alphas[alpha])
        cut = np.zeros((ntimes, 1), dtype="bool")
        dsigma = np.zeros((ntimes, 1), dtype="bool")
        for time in range(len(times)):
            psd1_naive_time = psd1_naive[time, :]
            psd1_slide_time = psd1_slide[time, :]
            psd2_naive_time = psd2_naive[time, :]
            psd2_slide_time = psd2_slide[time, :]

            naive_sensitivity_integrand_with_Hf = (
                calc_sens_integrand(
                    freq = freqs, P1 = psd1_naive_time, P2 = psd2_naive_time, window1 = window1, window2 = window2, delta_f = df, orf = orf, T = dt
                )
                / Hf**2
            )

            slide_sensitivity_integrand_with_Hf = (
                calc_sens_integrand(
                    freq = freqs, P1 = psd1_slide_time, P2 = psd2_slide_time, window1 = window1, window2 = window2, delta_f = df, orf = orf, T = dt
                )
                / Hf**2
            )
            naive_sigma_alpha = calc_sigma_alpha(
                sensitivity_integrand_with_Hf = naive_sensitivity_integrand_with_Hf[keep]
            )
            slide_sigma_alpha = calc_sigma_alpha(
                sensitivity_integrand_with_Hf = slide_sensitivity_integrand_with_Hf[keep]
            )

            cut[time], dsigma [time] = dsc_cut(naive_sigma = naive_sigma_alpha, slide_sigma = slide_sigma_alpha, dsc = dsc, bf_ss = bf_ss, bf_ns = bf_ns)

        cuts[alpha, :] = np.squeeze(cut)
        dsigmas[alpha, :] = np.squeeze(dsigma)

    for time in range(len(times)):
        overall_cut[time] = any(cuts[:, time])

    BadGPStimes = times[np.squeeze(overall_cut)]

    return BadGPStimes, dsigmas

In [9]:
pickle_path = "../test/test_data/naive_and_sliding_psds.pickle"

with open(pickle_path, "rb") as handle:
    pickle_loaded = pickle.load(handle)

naive_psd_1 = pickle_loaded["naive_psd_1"]
naive_psd_2 = pickle_loaded["naive_psd_2"]
avg_psd_1 = pickle_loaded["avg_psd_1"]
avg_psd_2 = pickle_loaded["avg_psd_2"]

In [10]:
run_dsc(
    dsc = 0.2,
    segment_duration = 192,
    sampling_frequency = 4096,
    psd1_naive = naive_psd_1,
    psd2_naive = naive_psd_2,
    psd1_slide = avg_psd_1,
    psd2_slide = avg_psd_2,
    alphas = [-5, 0, 3],
    orf = np.array([1]),
    notch_list_path = "../test/test_data/Official_O3_HL_notchlist.txt",
)

2024-02-28 13:42:34.295 | INFO     | __main__:run_dsc:23 - Running delta sigma cut


(array([1.24764440e+09, 1.24764449e+09, 1.24764459e+09]),
 array([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [11]:
shift = 1
a = np.array([1,2,3,4,5])
np.roll(a,shift)

array([5, 1, 2, 3, 4])

In [12]:
from numpy.random import random
def shift_timeseries(time_series_data: timeseries.TimeSeries, time_shift: int=0):
    if time_shift > 0:
        shifted_data = np.roll(time_series_data, shift)
    return shifted_data

t = timeseries.TimeSeries(random(10))

In [13]:
t

<TimeSeries([0.31373317, 0.76230909, 0.02091001, 0.58825514,
             0.48176435, 0.16966562, 0.58687714, 0.23191557,
             0.51443502, 0.93661302]
            unit=Unit(dimensionless),
            t0=<Quantity 0. s>,
            dt=<Quantity 1. s>,
            name=None,
            channel=None)>

In [14]:
shift_timeseries(t, 1)

<TimeSeries([0.93661302, 0.31373317, 0.76230909, 0.02091001,
             0.58825514, 0.48176435, 0.16966562, 0.58687714,
             0.23191557, 0.51443502]
            unit=Unit(dimensionless),
            t0=<Quantity 0. s>,
            dt=<Quantity 1. s>,
            name=None,
            channel=None)>

In [15]:
pickle_path='../test/test_data/H1L1_1247644138-1247645038.pickle'
with open(pickle_path, "rb") as handle:
            pickle_loaded = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../test/test_data/H1L1_1247644138-1247645038.pickle'

In [16]:
pickled_base = baseline.Baseline.load_from_pickle("../test/test_data/H1L1_1247644138-1247645038.pickle"
        )
pickled_ifo_1 = pickled_base.interferometer_1
pickled_ifo_2 = pickled_base.interferometer_2
naive_psd_1 = pickled_ifo_1.psd_spectrogram
naive_psd_2 = pickled_ifo_2.psd_spectrogram
avg_psd_1 = pickled_ifo_1.average_psd
avg_psd_2 = pickled_ifo_2.average_psd

FileNotFoundError: [Errno 2] No such file or directory: '../test/test_data/H1L1_1247644138-1247645038.pickle'